# Example:  Gauss Histogram Fitting 

This example shows how to fit an histogram using ROOT 

Create First an empty histogram with 64 bins with range [-8,8]

In [ ]:
TH1D h1("h1","h1",64,-8,8);

Fill the histogram with 1000 Gaussian Random number with mean=1 and sigma=2

In [ ]:
for (int i = 0; i < 1000; ++i) {
    h1.Fill(gRandom->Gaus(1,2));
}

Note, we could also use the function h1.FillRandom("gaus"), but we need to set the right parameters of the Gaussian function before.

We plot then the histogram (using JSROOT)

In [ ]:
%jsroot on
h1.DrawCopy();
gPad->Draw();

### Part1: Fit the Histogram with a Gaussian function

Note that before fitting we need to create the fitting function and set its initial parameter values. 
For a Gaussian function the setting of initial values is not really needed, because they can be computed automatically by ROOT. 

In [ ]:
// declare here some variables which will be used afterwards
TF1 * f1 = nullptr; 
TFitResultPtr res; 

Create the Gaussian Fit function

In [ ]:
f1 = new TF1("f1","gaus",-8,8);

Set the initial parametger values (e.g. Constant = 100, mean = 0, sigma =1)

In [ ]:
f1->SetParameters(100,0,1);  

Fit now the histogram using the Fit method in ROOT. 
- By default the least-square method is used. 
- For likelihood fits we need to use the option `"L"`
- for Pearson chi-square (expected error) , option `"P"`.

Use also  option `"S"` to create a TFitResult object that is returned to the user.
To compute the error using MINOS, use the `"E"` option


In [ ]:
ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");

In [ ]:
res  = h1.Fit(f1, "S   E");

In [ ]:
gStyle->SetOptFit(1111);
gPad->SetLogy(true); // to set log scale in y
gPad->Draw();

Print the result of the Fit from the returned TFitResult object

In [ ]:
res->Print();

We can plot the fit residuals using , see https://root.cern.ch/doc/master/classTRatioPlot.html 

In [ ]:
auto rp1 = new TRatioPlot(&h1);
rp1->Draw();
rp1->GetLowerRefYaxis()->SetTitle("ratio");
rp1->GetUpperRefYaxis()->SetTitle("entries");
gPad->Draw();

We can print also the Chisquare value of the fit. 
- if we have done a Neyman Chi2 fit we print the Chi2 computed using the bin values with observed errors (Neyman Chi2)
- if we have done a Likelihood fit  we compute a Chi2 from the Poisson likelihood (see this [paper](https://cds.cern.ch/record/148303?ln=en) from Baker-Cousins).

Note that in the first case the empty bins are not used, while in the second one all bins are used including empty ones. 

In [ ]:
//std::cout << "Neyman chi2             = " << h1.Chisquare(f1) << "/ NDF = " << f1->GetNDF() << std::endl;
std::cout << "Poisson likelihood chi2 = " << h1.Chisquare(f1,"L") << "/ NDF = " << h1.GetNbinsX()-f1->GetNpar() << std::endl;

Get the correlation matrix of the fit from the TFitResult class and print it 

In [ ]:
auto corrMatrix = res->GetCorrelationMatrix(); 
corrMatrix.Print(); 

In [ ]:
std::cout  << "Gaussian sigma = " << f1->GetParameter("Sigma") << "  +/- " << f1->GetParError(f1->GetParNumber("Sigma")) << std::endl;

If we want to access the MINOS asymmetric error, we can get them from the FitResult object

In [ ]:
std::cout  << "Gaussian sigma = " << res->Parameter(2) << "  + " << res->UpperError(2) << "  " << res->LowerError(2) << std::endl;